In [0]:
#Without Header
spark.sql("SELECT * FROM csv.`/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv`").display()

In [0]:
#Without Header using text
spark.sql("SELECT * FROM text.`/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv` ").display()

In [0]:
%sql
CREATE TEMPORARY VIEW netflix
USING csv
OPTIONS (
  path "/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv",
  header "true",
  inferSchema "true"
);

In [0]:
%sql
SELECT * FROM netflix;

In [0]:
df = spark.read.csv("/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv",header = True,inferSchema = True)
display(df)

In [0]:
%sql
DROP TABLE IF EXISTS workspace.default.netflix_viewrship;


In [0]:
df = spark.read.csv(
    "/Volumes/workspace/default/practice_dataset_da/BUS424-Netflix_Viewership.csv",
    header=True,
    inferSchema=True
)
new_cols = []
for c in df.columns:
    c=c.strip().lower().replace(' ','_')
    new_cols.append(c)
df = df.toDF(*new_cols)
df.write.mode("overwrite").saveAsTable("workspace.default.netflix_viewrship")


In [0]:
new_cols = []
for c in df.columns:
    c=c.strip().lower().replace(' ','_')
    new_cols.append(c)
print(new_cols)

In [0]:
%sql
DESCRIBE EXTENDED workspace.default.netflix_viewrship

In [0]:
%sql
DESCRIBE HISTORY workspace.default.netflix_viewrship